In [1]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers, datasets


In [2]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_dataset(base_dir, target_size=(224, 224)):
    images = []
    labels = []
    class_labels = {'hearts': 0, 'diamonds': 1, 'clubs': 2, 'spades': 3}
    
    # Navigate through each suit folder
    for suit in class_labels.keys():
        suit_dir = os.path.join(base_dir, suit)
        print(f"Checking suit directory: {suit_dir}")  # Debug print
        if os.path.isdir(suit_dir):
            label_index = class_labels[suit]
            print(f"Label index for {suit}: {label_index}")  # Debug print

            # Navigate through each card folder within the suit folder
            for card_folder in os.listdir(suit_dir):
                card_folder_path = os.path.join(suit_dir, card_folder)
                #print(f"Checking card folder: {card_folder_path}")  # Add this debug print
                if os.path.isdir(card_folder_path):
                    # Load each image from the card folder
                    for image_file in os.listdir(card_folder_path):
                        image_path = os.path.join(card_folder_path, image_file)
                        image = load_img(image_path, target_size=target_size)
                        #print(f"Loading image: {image_path}")  # Debug print
                        image_array = img_to_array(image)
                        images.append(image_array)
                        labels.append(label_index)
    
    # After loading all images and labels, shuffle them
    combined = list(zip(images, labels))
    np.random.shuffle(combined)
    images[:], labels[:] = zip(*combined)                    
    
    images = np.array(images, dtype='float32') / 255.0  # Normalize to [0, 1]
    labels = np.array(labels, dtype='int32')
    
    # Print some of the labels to verify they are correct
    print(labels[:100])
    
    return images, labels, class_labels

# Directory Definitions
base_dir = "c:\\Users\\alec\\OneDrive - University of North Georgia\\FALL 2023\\Cyber capstone\\Experimentation\\SortedData"
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

# Usage
train_images, train_labels, train_class_labels = load_dataset(train_dir)
valid_images, valid_labels, valid_class_labels = load_dataset(validation_dir)
test_images, test_labels, test_class_labels = load_dataset(test_dir)

# Check shapes and class labels
print(train_images.shape, train_labels.shape)
print(valid_images.shape, valid_labels.shape)
print(test_images.shape, test_labels.shape)
print(train_class_labels)


Checking suit directory: c:\Users\alec\OneDrive - University of North Georgia\FALL 2023\Cyber capstone\Experimentation\SortedData\train\hearts
Label index for hearts: 0
Checking suit directory: c:\Users\alec\OneDrive - University of North Georgia\FALL 2023\Cyber capstone\Experimentation\SortedData\train\diamonds
Label index for diamonds: 1
Checking suit directory: c:\Users\alec\OneDrive - University of North Georgia\FALL 2023\Cyber capstone\Experimentation\SortedData\train\clubs
Label index for clubs: 2
Checking suit directory: c:\Users\alec\OneDrive - University of North Georgia\FALL 2023\Cyber capstone\Experimentation\SortedData\train\spades
Label index for spades: 3
[3 3 2 0 1 0 3 0 2 2 1 3 0 3 1 1 3 2 3 2 0 0 2 0 3 1 0 0 2 0 0 1 3 1 3 1 2
 1 3 3 3 0 0 0 2 3 3 3 0 0 3 0 3 0 1 0 0 3 2 0 1 3 1 1 1 3 3 3 3 2 1 0 1 0
 2 3 2 3 0 1 3 0 1 3 3 3 1 2 1 3 3 3 0 3 1 3 2 2 0 1]
Checking suit directory: c:\Users\alec\OneDrive - University of North Georgia\FALL 2023\Cyber capstone\Experimentation

In [3]:
# Shuffle the training dataset
train_indices = np.arange(train_images.shape[0])
np.random.shuffle(train_indices)
train_images = train_images[train_indices]
train_labels = train_labels[train_indices]

# Shuffle the validation dataset
valid_indices = np.arange(valid_images.shape[0])
np.random.shuffle(valid_indices)
valid_images = valid_images[valid_indices]
valid_labels = valid_labels[valid_indices]

# Optionally, if you also want to shuffle the test dataset:
test_indices = np.arange(test_images.shape[0])
np.random.shuffle(test_indices)
test_images = test_images[test_indices]
test_labels = test_labels[test_indices]

In [4]:
#/////IGNORE DATA AUGMENTATION///// 
#/////NOT NEEDED FOR TRAINING//////

#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Data augmentation
#data_gen = ImageDataGenerator(
#    rotation_range=20,
#    width_shift_range=0.2,
#    height_shift_range=0.2,
#    shear_range=0.2,
#    zoom_range=0.2,
#    horizontal_flip=True,
#    fill_mode='nearest'
#)


In [4]:
print(train_images.shape, train_labels.shape)
print(valid_images.shape, valid_labels.shape)
print(test_images.shape, test_labels.shape)

(7509, 224, 224, 3) (7509,)
(260, 224, 224, 3) (260,)
(260, 224, 224, 3) (260,)


In [5]:
print(train_labels[:100])

[1 1 2 2 2 1 1 3 1 2 3 0 3 0 0 2 2 2 2 0 0 0 0 0 0 2 3 1 2 1 0 3 3 0 0 1 3
 1 3 3 1 1 3 0 1 3 3 0 3 3 0 0 1 2 3 1 2 3 2 2 1 1 1 0 1 1 2 3 0 2 0 3 0 2
 0 3 1 0 1 1 2 2 0 3 0 2 2 2 3 2 3 1 1 3 2 2 0 1 2 0]


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SeparableConv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2

input_shape = (224, 224, 3)

model = Sequential([
    # Increase the number of filters and add depthwise separable convolutions
    SeparableConv2D(64, (3, 3), activation='relu', input_shape=input_shape, padding='same', depth_multiplier=1, kernel_regularizer=l2(0.001)),
    MaxPooling2D((2, 2)),
    Dropout(0.1),

    SeparableConv2D(128, (3, 3), activation='relu', padding='same', depth_multiplier=1, kernel_regularizer=l2(0.001)),
    MaxPooling2D((2, 2)),
    Dropout(0.1),

    SeparableConv2D(256, (3, 3), activation='relu', padding='same', depth_multiplier=1, kernel_regularizer=l2(0.001)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),  # Increased number of units in the dense layer
    Dropout(0.25),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 separable_conv2d (Separabl  (None, 224, 224, 64)      283       
 eConv2D)                                                        
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 112, 112, 64)      0         
                                                                 
 separable_conv2d_1 (Separa  (None, 112, 112, 128)     8896      
 bleConv2D)                                                      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                   

In [7]:
history = model.fit(
    #data_gen.flow(train_images, train_labels, batch_size=32),
    train_images, train_labels,  # Use the original images and labels without augmentation
    batch_size=32,
    epochs=30,
    validation_data=(valid_images, valid_labels)
    
)

Epoch 1/30
235/235 [==============================] - 297s 1s/step - loss: 1.2762 - accuracy: 0.3597 - val_loss: 1.0086 - val_accuracy: 0.4731
Epoch 2/30
235/235 [==============================] - 308s 1s/step - loss: 1.0158 - accuracy: 0.5058 - val_loss: 0.9302 - val_accuracy: 0.5654
Epoch 3/30
235/235 [==============================] - 309s 1s/step - loss: 0.9320 - accuracy: 0.5968 - val_loss: 0.7631 - val_accuracy: 0.7269
Epoch 4/30
235/235 [==============================] - 311s 1s/step - loss: 0.8397 - accuracy: 0.7217 - val_loss: 0.5494 - val_accuracy: 0.8692
Epoch 5/30
235/235 [==============================] - 306s 1s/step - loss: 0.7449 - accuracy: 0.7833 - val_loss: 0.5576 - val_accuracy: 0.8538
Epoch 6/30
235/235 [==============================] - 296s 1s/step - loss: 0.6778 - accuracy: 0.8104 - val_loss: 0.5423 - val_accuracy: 0.8769
Epoch 7/30
235/235 [==============================] - 296s 1s/step - loss: 0.6353 - accuracy: 0.8362 - val_loss: 0.5052 - val_accuracy: 0.8923

In [8]:
model.save('PlayingCardHardened.h5')

C:\Users\alec\miniconda3\envs\jupyterenv\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
